<a href="https://colab.research.google.com/github/JamesKha/MovieRecommendation/blob/main/TfIdfVectorizer_Checker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

In [2]:
tagsDF = pd.read_csv('/content/drive/MyDrive/Miniproject Files/tags.csv')
moviesDF = pd.read_csv('/content/drive/MyDrive/Miniproject Files/movies.csv')

In [3]:
tagsAndGenresDataFrame = pd.merge(tagsDF, moviesDF, on='movieId')[['movieId','title', 'tag','genres']]
titlesAndTags = pd.DataFrame(tagsAndGenresDataFrame.groupby(['title']).tag.unique())
titlesAndTags = titlesAndTags.reset_index()
titlesAndTags['tag'] = titlesAndTags['tag'].apply(lambda x: ' '.join([str(i) for i in x]))
titlesAndTags

,title,tag
0,(500) Days of Summer (2009),artistic Funny humorous inspiring intelligent ...
1,...And Justice for All (1979),lawyers
2,10 Cloverfield Lane (2016),creepy suspense
3,10 Things I Hate About You (1999),Shakespeare sort of
4,101 Dalmatians (1996),dogs remake
...,...,...
1567,Zero Dark Thirty (2012),Afghanistan American propaganda assassination ...
1568,Zombieland (2009),Bill Murray dark comedy Emma Stone funny Jesse...
1569,Zoolander (2001),ben stiller comedy David Bowie goofy mindless ...
1570,Zulu (1964),Africa


In [4]:
titlesAndTags.head()

,title,tag
0,(500) Days of Summer (2009),artistic Funny humorous inspiring intelligent ...
1,...And Justice for All (1979),lawyers
2,10 Cloverfield Lane (2016),creepy suspense
3,10 Things I Hate About You (1999),Shakespeare sort of
4,101 Dalmatians (1996),dogs remake


In [5]:
tfidf = TfidfVectorizer(stop_words='english')
titlesAndTags['tag'] = titlesAndTags['tag'].fillna('')
tfidf_matrix = tfidf.fit_transform(titlesAndTags['tag'])

In [6]:
from sklearn.metrics.pairwise import linear_kernel
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [7]:
indices = pd.Series(titlesAndTags.index, index=titlesAndTags['title']).drop_duplicates()

In [8]:
indices

title
(500) Days of Summer (2009)             0
...And Justice for All (1979)           1
10 Cloverfield Lane (2016)              2
10 Things I Hate About You (1999)       3
101 Dalmatians (1996)                   4
                                     ... 
Zero Dark Thirty (2012)              1567
Zombieland (2009)                    1568
Zoolander (2001)                     1569
Zulu (1964)                          1570
eXistenZ (1999)                      1571
Length: 1572, dtype: int64

In [9]:
cosine_sim

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [17]:
def get_recommendations(title, cosine_sim=cosine_sim):

    idx = indices[title]

    sim_scores = list(enumerate(cosine_sim[idx]))


    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)


    sim_scores = sim_scores[1:11]


    movie_indices = [i[0] for i in sim_scores]


    return titlesAndTags[['title','tag']].iloc[movie_indices]

In [18]:
titlesAndTags

,title,tag
0,(500) Days of Summer (2009),artistic Funny humorous inspiring intelligent ...
1,...And Justice for All (1979),lawyers
2,10 Cloverfield Lane (2016),creepy suspense
3,10 Things I Hate About You (1999),Shakespeare sort of
4,101 Dalmatians (1996),dogs remake
...,...,...
1567,Zero Dark Thirty (2012),Afghanistan American propaganda assassination ...
1568,Zombieland (2009),Bill Murray dark comedy Emma Stone funny Jesse...
1569,Zoolander (2001),ben stiller comedy David Bowie goofy mindless ...
1570,Zulu (1964),Africa


In [19]:
get_recommendations('Step Brothers (2008)')

,title,tag
986,Old School (2003),comedy Will Ferrell
66,Anchorman 2: The Legend Continues (2013),comedy Steve Carell stupid but funny will ferrell
36,Airheads (1994),Comedy
242,Chalet Girl (2011),Comedy
146,"Big Lebowski, The (1998)",Coen Brothers black comedy bowling classic coe...
491,Game Night (2018),Comedy funny Rachel McAdams
67,Anchorman: The Legend of Ron Burgundy (2004),hilarious Steve Carell Will Ferrell stupid awe...
274,Clueless (1995),chick flick funny Paul Rudd quotable seen more...
713,Jumanji: Welcome to the Jungle (2017),Action Comedy Dwayne Johnson funny
1385,The Lego Batman Movie (2017),funny heartwarming
